In [ ]:
import scipy.io as scio
import numpy as np    
import matplotlib.pyplot as plt
import sys
import os
import math
import pprint
import cv2
from scipy.misc import imsave
from helper import *
from create_labels import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Applying Weighting algorithm to an image
value = 1
def weighting_algo(id_image):
    '''
    Creating weighting of an image
    '''
    image = id_image
    x,y = image.shape
    weighted_image = np.zeros((x,y))
    for j in range(x):
        for k in range(y):
            # NOTE: did weights based on bold from legend
            if(image[j][k]==1): # 1 red - myocardium
                 w2 = 9 * value 
            elif(image[j][k] == 2): # blue - endocardium
                 w2 = 10 * value
            elif(image[j][k]== 3): # purple - fibrosis
                 w2 = 4 * value 
            elif(image[j][k] == 4): # green - fat
                 w2 = 11 * value
            elif(image[j][k]== 5): # orange - dense collagen
                 w2 = 5 * value
            elif(image[j][k]== 6): # yellow - loose collagen
                 w2 = 5 * value
            elif(image[j][k]== 7):
                 w2 = 6 * value # magenta - smooth muscle
            else:
                 w2 = 0
            if(j!=0 and j!=x-1):
                next_pix = image[j+1][k]
                prev_pix = image[j-1][k]
                # Taking the derivative of the pixels
                if(np.absolute((next_pix-prev_pix))>0 and w2!=0):
                    w1 = 10  
                else:
                    w1 = 0
            else:
                w1 = 0
            w = 1 + w1 + w2
            weighted_image[j][k] = w
    return weighted_image

#### Make Segments

In [ ]:
# 0 black
# 1 red - myocardium
# 2 blue - endocardium
# 3 purple - fibrosis
# 4 green - fat
# 5 orange - dense collagen
# 6 yellow - loose collagen
# 7 magenta - smooth muscle

In [ ]:
# Setting the directories
import os

# wanted_folder = 'alldata/'
# wanted_folder = 'pruned/'
# wanted_folder = 'Atrium/'
wanted_folder = 'Ventricle/'

cwd = os.getcwd()
check_directory = cwd
if check_directory == '/home/sim/notebooks/relaynet_pytorch':
    cwd = cwd + '/datasets/OCTData/'+wanted_folder
elif check_directory == '/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch':
    cwd = cwd + '/datasets-24-aug/OCTData/'+wanted_folder

print(cwd)

In [ ]:
# Raw Image
whole_raw_image_folder = cwd + 'whole_raw_image/'
brushlet_enhanced_folder = cwd + 'brushlet_enhanced/'
brushlet_denoised_folder = cwd + 'brushlet_denoised/'

# Labels
manual_label_folder = cwd + 'manual_label/'
all_labels_folder = cwd + 'all_labels/'
sim_labels_folder = cwd + 'my_labels/'
corrected_labels_folder = cwd + 'png_labels_method/'

# Normalised Images
normalised_folder = cwd + 'normalised_raw_image/'

list_of_folders = [ whole_raw_image_folder, \
                    brushlet_denoised_folder, \
                    brushlet_enhanced_folder, \
                    manual_label_folder, \
                    all_labels_folder, \
                    sim_labels_folder, \
                    corrected_labels_folder, \
                    normalised_folder
                  ]

# Denoised Images
# brushlet_enhanced_folder = cwd + 'normalised_brushlet_enhanced/'
# brushlet_denoised_folder = cwd + 'normalised_brushlet_denoised/'
# bm3d_folder = cwd + 'normalised_bm3d/'
# nlm_folder = cwd + 'normalised_nl_means/'
# bilateral_folder = cwd + 'normalised_bilateral_filter/'

# # Denoised Images
# brushlet_enhanced_folder = cwd + 'normalised_brushlet_enhanced/'
# brushlet_denoised_folder = cwd + 'normalised_brushlet_denoised/'
# bm3d_folder = cwd + 'normalised_bm3d/'
# nlm_folder = cwd + 'normalised_nl_means/'
# bilateral_folder = cwd + 'normalised_bilateral_filter/'

# list_of_folders = [ whole_raw_image_folder, \
#                     brushlet_denoised_folder, \
#                     brushlet_enhanced_folder, \
#                     manual_label_folder, \
#                     all_labels_folder, \
#                     sim_labels_folder, \
#                     corrected_labels_folder, \
#                     normalised_folder,  \
#                     bm3d_folder,\
#                     nlm_folder,\
#                     bilateral_folder
#                   ]
print(len(list_of_folders))
def list_all_files(directory):
    from os import listdir
    from os.path import isfile, join
    all_labels_files = [f for f in listdir(directory) if isfile(join(directory, f)) and f != '.DS_Store']
    all_labels_files.sort()
    return all_labels_files

files_list = []
for folders in list_of_folders:
    files_list.append(list_all_files(folders))

In [ ]:
print(files_list[0])

In [ ]:
# Creating list of file names
new_file_list = []

for i in files_list[0]:
    if i[:4] == 'con_':
        string = i[4:]
        file_name = string[:19].replace('.','')
    new_file_list.append(file_name)

new_file_list.sort()

In [ ]:
print(len(new_file_list))

In [ ]:
# Removing Unwanted Files
unwanted_files = []
test_set = ["H1_N01848_LA_1_272","H2_N02047_RVS_1_400","H3_N02186_LA_1_400","H3_N02186_LVS_1_676","H4_N03210_RA_1_400","H5_N03290_LV_1_420","H6_N03320_LVS_1_400","H6_N03320_LVS_1_455","H7_N03555_LVS_1_306","H7_N03555_RA_1_302","H8_N03585_RVS_1_398","H8_N03585_RA_1_345","H9_N03857_LV_1_580"]
print(len(test_set))
not_in = []
for count, i in enumerate(new_file_list):
    if i == "H1_N01848_LA_1_272":
        print(i)
    if i in test_set:
        print(count,i)
        not_in.append(i)
        unwanted_files.append(count)
print(unwanted_files)

In [ ]:
print(len(unwanted_files))
print(len(new_file_list))
print(len(new_file_list)-len(unwanted_files))

In [ ]:
# Booosting underseperatted 
underrepresented = ["H1_N01848_LA_1_272","H1_N01848_LA_1_388","H2_N02047_LA_1_241","H2_N02047_LA_1_257","H2_N02047_RA_1_380","H3_N02186_LVS_1_676","H3_N02186_RA_1_380","H3_N02186_RA_1_40","H4_N03210_LVS_1_40","H5_N03290_LA_1_380","H5_N03290_LA_1_400","H6_N03320_LA_1_450"]
print(len(underrepresented))

### Saving Image Helpers

In [ ]:
def segment_image_multiple(image, left_bound, right_bound, split):
    '''
    Segment image at intervals of half of split to have more training data
    
    Segment image by input of image, left bound and right bound, and split int 
    '''
    list_of_images = []
    bounds = right_bound - left_bound
    quot, rem = divmod(bounds, split)
    ranged = np.arange(0,quot,0.25) # quadruple amount of data
    ranged = ranged[:-2] 
    for i in ranged:
        if len(image.shape) == 2:
            cropped_image = crop_image(image, int(i*split), int((i+1) * split))
            list_of_images.append(cropped_image)
        elif len(image.shape) == 3:
            cropped_image = crop_image(image, int(i*split), int((i+1) * split), False)
            list_of_images.append(cropped_image)
    return list_of_images

whole_raw_1 = plt.imread(whole_raw_image_folder+files_list[0][0])
segmented_whole = segment_image_multiple(whole_raw_1, 0, 600, 64)

In [ ]:
# Saving images
def crop_and_save_raw(image, src_direct, name):
    dst_drc = src_direct + 'Train/'

    # Crop all images
    if len(image.shape) == 2:
        height, width = image.shape
        if width > 600:
            image = crop_image(image, 0, 600) # Crop all images at 512,600
    elif len(image.shape) == 3:
        height, width, colour = image.shape
        if width > 600:
            image = crop_image(image, 0, 600, colour) # Crop all images at 512,600
    segmented_images = segment_image_multiple(image, 0, 600, 64)
    
#     print(len(segmented_images))
#     fig, axes = plt.subplots(nrows=1, ncols=len(segmented_images), figsize=(9,20))
#     for i, ax in enumerate(axes):
#         ax.imshow(segmented_images[i])
#         ax.set_title(i)
#     plt.show()
    # If in underrepresented, double image amount by horizontally flipping and adding to dataset
    if name in underrepresented:
        for i in range(len(segmented_images)):
            image_name = dst_drc+name+'_'+str(i+1)+'.png'
            cur_image = segmented_images[i]
            imsave(image_name, cur_image)
            
            repeat_name = dst_drc+name+'_'+str(i+1+len(segmented_images))+'.png'
            flipped_image = np.fliplr(segmented_images[i])
            imsave(repeat_name, flipped_image)
    else:
        for i in range(len(segmented_images)):
            image_name = dst_drc+name+'_'+str(i+1)+'.png'
            cur_image = segmented_images[i]
            imsave(image_name, cur_image)

# Saving labels and weights
def crop_and_save_label(label, src_direct, name):
    id_dst_drc = src_direct + 'Train/segmented_ids/'
    weight_dst_drc = src_direct + 'Train/weights/'
    height, width = label.shape
    
    label[440:,:] = 0
    
    if width > 600:
        label = crop_image(label, 0, 600)
    if height != 512:
        amount = 512 - height
        id_full = np.zeros((amount,600))
        label = np.concatenate((label, id_full))
    
    rgb_image = convert_to_rgb_image(label) 
    segmented_labels = segment_image_multiple(label, 0, 600, 64)
    
    # If in underrepresented, double image amount by horizontally flipping
    if name in underrepresented:
        for i in range(len(segmented_labels)):
            id_name = id_dst_drc+'id_'+name+'_'+str(i+1)+'.npy'
            weight_name = weight_dst_drc+'weight_'+name+'_'+str(i+1)+'.npy'
            
            # convert image_id into weighted image 
            cur_label = segmented_labels[i]
            weighted_image = weighting_algo(cur_label)
            
            # Saving labels and weights
            np.save(id_name,np.array(cur_label))
            np.save(weight_name,np.array(weighted_image))
            
            repeat_id_name = id_dst_drc+'id_'+name+'_'+str(i+1+len(segmented_labels))+'.npy'
            repeat_weight_name = weight_dst_drc+'weight_'+name+'_'+str(i+1+len(segmented_labels))+'.npy'
            
            # convert image_id into weighted image 
            flipped_label = np.fliplr(segmented_labels[i])
            flipped_weight = np.fliplr(weighting_algo(cur_label))
            
            # Saving labels and weights
            np.save(repeat_id_name,np.array(flipped_label))
            np.save(repeat_weight_name,np.array(flipped_weight))
    else:
        for i in range(len(segmented_labels)):
            id_name = id_dst_drc+'id_'+name+'_'+str(i+1)+'.npy'
            weight_name = weight_dst_drc+'weight_'+name+'_'+str(i+1)+'.npy'
            
            # convert image_id into weighted image 
            cur_label = segmented_labels[i]
            weighted_image = weighting_algo(cur_label)

            # Saving labels and weights
            np.save(id_name,np.array(cur_label))
            np.save(weight_name,np.array(weighted_image))

    return name

# Saving images
def save_label(image, src_direct, name):
    dst_drc = src_direct
    
    # Crop all images
    if len(image.shape) == 2:
        height, width = image.shape
        if width > 600:
            image = crop_image(image, 0, 600) # Crop all images at 512,600
    elif len(image.shape) == 3:
        height, width, colour = image.shape
        if width > 600:
            image = crop_image(image, 0, 600, colour) # Crop all images at 512,600
    id_image = convert_to_id_image(image)
    image_name = dst_drc+"label_"+name+'.png'
    imsave(image_name, id_image)
#     rgb_image = convert_to_rgb_image(id_image)
#     plt.imshow(id_image)
#     plt.imshow(rgb_image)
#     print(id_image.shape)
#     print(np.unique(id_image))
#     return image_name

In [ ]:
def remove_files(mydir, ext):
    filelist = [ f for f in os.listdir(mydir) if f.endswith(ext) ]
    for f in filelist:
        os.remove(os.path.join(mydir, f))
remove = True
if remove:
    whole_raw_image_folder_rem = whole_raw_image_folder + 'Train/'
    brushlet_enhanced_folder_rem = brushlet_enhanced_folder +'Train/'
    brushlet_denoised_folder_rem = brushlet_denoised_folder+'Train/'
    manual_label_folder_rem = manual_label_folder+'Train/'
    
    all_labels_folder_ids_rem = all_labels_folder + 'Train/segmented_ids/'
    all_labels_folder_weights_rem = all_labels_folder +'Train/weights/'
   
    sim_labels_folder_ids_rem = sim_labels_folder + 'Train/segmented_ids/'
    sim_labels_folder_weights_rem = sim_labels_folder + 'Train/weights/'
    
    corrected_labels_folder_ids_rem = corrected_labels_folder + 'Train/segmented_ids/'
    corrected_labels_folder_weights_rem = corrected_labels_folder + 'Train/weights/'
    
    normalised_folder_rem = normalised_folder + 'Train/'
    
#     bm3d_folder_rem = bm3d_folder + 'Train/'
#     nlm_folder_rem = nlm_folder + 'Train/'
#     bilateral_folder_rem = bilateral_folder + 'Train/'

    
    remove_files(whole_raw_image_folder_rem, '.png')
    remove_files(brushlet_enhanced_folder_rem, '.png')
    remove_files(brushlet_denoised_folder_rem, '.png')
    remove_files(manual_label_folder_rem, '.png')
    remove_files(all_labels_folder_ids_rem, '.npy')
    remove_files(all_labels_folder_weights_rem, '.npy')
    remove_files(sim_labels_folder_ids_rem, '.npy')
    remove_files(sim_labels_folder_weights_rem, '.npy')
    remove_files(corrected_labels_folder_ids_rem, '.npy')
    remove_files(corrected_labels_folder_weights_rem, '.npy')
    remove_files(normalised_folder_rem, '.png')
#     remove_files(bm3d_folder_rem, '.png')
#     remove_files(nlm_folder_rem, '.png')
#     remove_files(bilateral_folder_rem, '.png')

In [ ]:
# The values of the conversion by Elsa
values = [0.0, 0.007843138, 0.011764706, 0.015686275, 0.019607844, 0.023529412, 0.02745098]

In [ ]:
# Extensions
raw_ext = '.tif'
label_ext = '.JPG'

whole_raw_ext = '.tif'
man_lab_ext = '.JPG'
everything_else_ext = '.png'

for i in range(len(files_list[0])):
    if i % 2 == 0:
        print(i)
        whole_raw = plt.imread(whole_raw_image_folder+files_list[list_of_folders.index(whole_raw_image_folder)][i]) # whole raw image - needs to be segmented and saved
        
        # Labels
        man_lab = plt.imread(manual_label_folder+files_list[list_of_folders.index(manual_label_folder)][i]) # main_label (coloured) 
        correct_label = plt.imread(corrected_labels_folder+'label_'+new_file_list[i]+'_labels.png') # label (ids) 
        correct_label = ((correct_label*7)/np.max(values)).astype(int)
        
        # Denoised
#         b_d = plt.imread(brushlet_denoised_folder+files_list[list_of_folders.index(brushlet_denoised_folder)][i]) # whole brushlet denoised image 
#         b_e = plt.imread(brushlet_enhanced_folder+files_list[list_of_folders.index(brushlet_enhanced_folder)][i]) # whole brushlet enhanced image
#         bm3d = plt.imread(bm3d_folder+files_list[list_of_folders.index(bm3d_folder)][i]) # bm3d image 
#         nlm = plt.imread(nlm_folder+files_list[list_of_folders.index(nlm_folder)][i]) # nlm image
#         bilateral = plt.imread(bilateral_folder+files_list[list_of_folders.index(bilateral_folder)][i]) # bilateral image 
        
        # Normalised
        normalised = plt.imread(normalised_folder+files_list[7][i])
        
        #### crop and save all files necessary
        crop_and_save_raw(whole_raw, whole_raw_image_folder, new_file_list[i])
        
        crop_and_save_raw(man_lab, manual_label_folder, new_file_list[i])
        crop_and_save_label(correct_label, corrected_labels_folder, new_file_list[i])
        
        # Create my labels from manual labels
        # label = plt.imread(all_labels_folder+files_list[4][i]) # label (ids) 
        # label = ((label*7)/np.max(values)).astype(int)
        # save_label(man_lab, sim_labels_folder, new_file_list[i])
        # sim_label = plt.imread(sim_labels_folder+files_list[5][i]) # label (ids)
        # crop_and_save_label(label, all_labels_folder, new_file_list[i])
        # crop_and_save_label(sim_label, sim_labels_folder, new_file_list[i])
        
        
        crop_and_save_raw(normalised, normalised_folder, new_file_list[i])
        
        
#         crop_and_save_raw(b_d, brushlet_denoised_folder, new_file_list[i])
#         crop_and_save_raw(b_e, brushlet_enhanced_folder, new_file_list[i])
#         crop_and_save_raw(bm3d, bm3d_folder, new_file_list[i])
#         crop_and_save_raw(nlm, nlm_folder, new_file_list[i])
#         crop_and_save_raw(bilateral, bilateral_folder, new_file_list[i])

        # image_array = [b_d, b_e, bm3d, nlm, bilateral]    
        # image_name = ['b_d', 'b_e', 'bm3d', 'nlm', 'bilateral']

        # fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(15,5))
        # fig.suptitle(new_file_list[i], size=14)
        # for j, ax in enumerate(axes):
        #    ax.margins(0.05, 0.15)
        #    ax.imshow(image_array[j])
        #    ax.set_title(image_name[j])

In [ ]:
print(files_list[0][15])

In [ ]:
# # for i in range(len(files_list[0])):
# for i in range(2):
#     if i not in unwanted_files:
#         man_lab = plt.imread(manual_label_folder+files_list[3][i])
#     #     correct_label = plt.imread(corrected_labels_folder+files_list[6][i])
#         correct_label = plt.imread(corrected_labels_folder+'label_'+new_file_list[i]+'_labels.png')
#         correct_label = ((correct_label*7)/np.max(values)).astype(int)

#         rgb_image = convert_to_rgb_image(correct_label)

#         plt.figure(figsize=(15,5))
#         plt.imshow(man_lab)
#         plt.suptitle(new_file_list[i], size=15)

#         print()
#         print(new_file_list[i])
#         print(np.unique(correct_label))

#         plt.figure(figsize=(15,5))
#         plt.imshow(correct_label)

#         plt.figure(figsize=(15,5))
#         plt.imshow(rgb_image)
#         plt.show()
#         h,w = correct_label.shape
#         print(correct_label[400])

# Create IMDB of dataset - HP5

In [ ]:
# import numpy as np
# import h5py
# import fnmatch

In [ ]:
# files = list_all_files(whole_raw_image_folder+'Train/')
# H = 512
# W = 64
# N = len(files)
# print(N)

In [ ]:
# # atrium_ids = all_labels_folder+'Train/segmented_ids/'
# # atrium_weights = all_labels_folder+'Train/weights/'
# atrium_ids = corrected_labels_folder+'Train/segmented_ids/'
# atrium_weights = corrected_labels_folder+'Train/weights/'
# atrium_raw = whole_raw_image_folder +'Train/'

# atrium_id_files = {}
# atrium_weights_files = {}
# atrium_raw_files = {}
# for number, filename in enumerate(sorted(os.listdir(atrium_ids)), start=1):
#     atrium_id_files[number] = filename
# for number, filename in enumerate(sorted(os.listdir(atrium_weights)), start=1):
#     atrium_weights_files[number] = filename
# for number, filename in enumerate(sorted(os.listdir(atrium_raw)), start=1):
#     atrium_raw_files[number] = filename

In [ ]:
# # Load h5py raw array
# def make_h5py_array_raw(src_direct, dictionary, count):
#     files = []
#     for i in range(1,len(dictionary)+1):
#         act_file = plt.imread(src_direct+dictionary[i])
#         act_file = np.array(act_file)
#         files.append(act_file)
#         count += 1
#     return files, count

# # Load h5py py array
# def make_h5py_array_npy(src_direct, dictionary, count):
#     files = []
#     for i in range(1,len(dictionary)+1):
#         act_file = np.load(src_direct+dictionary[i])
#         files.append(act_file)
        
#         count += 1
#     return files, count
# count = 0
# id_files,num = make_h5py_array_npy(atrium_ids, atrium_id_files, count)
# print(len(id_files),num)
# weight_files,num = make_h5py_array_npy(atrium_weights, atrium_weights_files, count)
# print(len(weight_files),num)
# raw_files, num = make_h5py_array_raw(atrium_raw, atrium_raw_files, count)
# print(len(raw_files),num)

In [ ]:
# import torch

# # Creating Data.h5 File
# data = np.zeros((N,1, H, W), dtype=np.float32)


# for i in range(N):
#     image = raw_files[i] # array of size (H,W)
#     for m in range(H):
#         for n in range(W):
#             data[i, 0, m, n] = image[m,n]
# data = data.astype('float32')
# print(type(data))

# # data = torch.from_numpy(data).float()
# data = torch.from_numpy(data)
# # data = torch.Tensor(data)
# print('Finalshape:',data.shape)
# hf = h5py.File('/home/sim/notebooks/relaynet_pytorch/datasets/OCTData/Data.h5', 'w')
# hf.create_dataset('data', data=data) # creating raw data dataset - group name followed by dimensions in [H,W,Channel,DataIndex]
# hf.close()

In [ ]:
# # Creating label.h5 File
# labels = np.zeros((N,2, H, W), dtype=np.float32)

# for i in range(N):
#     weights = weight_files[i] # array of size (H,W)
#     ids = id_files[i] # array of size (H,W) # class is your colour
    
#     h,w = ids.shape
#     if h != 512 or w != 64:
# #         print(h,w)
#         amount = H - h
#         weight_full = np.full((amount, 64), 1.0)
#         id_full = np.full((amount, 64), 7)
#         weights = np.concatenate((weights, weight_full))
#         ids = np.concatenate((ids, id_full))   
#     new_id = np.copy(ids)
#     for m in range(H):
#         for n in range(W):
#             if ids[m,n] > 7:
#                 # set all label values to 7 
#                 new_id[m,n] = 7
#             labels[i, 0, m, n] = new_id[m,n]
#             labels[i, 1, m, n] = weights[m,n]
            
# labels = labels.astype('float32')    
# print(type(labels))
# # labels = torch.from_numpy(labels).float()
# labels = torch.from_numpy(labels)
# # print(np.unique(labels[:,0,:,:]))
# print('Finalshape:',labels.shape)

# hf = h5py.File('/home/sim/notebooks/relaynet_pytorch/datasets/OCTData/label.h5', 'w')
# hf.create_dataset('labels', data=labels) 
# hf.close()

In [ ]:
# # Creating set.h5 File
# sets = np.ones((1,N))
# split = N * 0.8

# num_files = N
# validation_cutoff = int(split)

# # Making training set random
# train_indices = np.random.choice(num_files,validation_cutoff,replace = False)
# print(train_indices)
# test_indices = [x for x in range(num_files) if x not in train_indices]
# print(test_indices)
# for count,i in enumerate(sets[0]):
#     if count not in train_indices:
#         sets[0][count] = 3

# # sets[:,int(split):] = 3

# hf = h5py.File('/home/sim/notebooks/relaynet_pytorch/datasets/OCTData/set.h5', 'w')
# hf.create_dataset('set', data=sets)
# hf.close()